1. Acceso a la página de La Casa del Libro: Utilizando Selenium, deberás automatizar el proceso de navegación hacia la página de libros de ficción en el sitio web de La Casa del Libro.
2. Extracción de datos de las 5 primeras páginas: Una vez en la página de libros de ficción, debes programar su script para que recorra los 5 primeros libros, extrayendo los siguientes datos de cada uno:
    - Título del libro
    - Autor(es) del libro
    - Precio del libro
    - Editorial del libro
    - Idioma del libro
    - Número de páginas del libro
3. Almacenamiento de datos: Los datos extraídos deben almacenarse en un DataFrame.

In [1]:
import pandas as pd 

import numpy as np

from bs4 import BeautifulSoup
import requests

from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select  

from time import sleep  

pd.set_option('display.max_columns', None)

In [3]:
driver = webdriver.Chrome()

driver.get('https://www.casadellibro.com/')

driver.maximize_window()

sleep(3)

try:
    # Rechaza las cookies
    driver.find_element('css selector' , '#onetrust-reject-all-handler').click()
    
except:
    pass

sleep(3)

# Accede a los libros de ficción en el menú
driver.find_element('css selector' , '#app > div.v-application--wrap > div.when-mobile > div.cabecera.desktop > div.bottom-row > div > div > div:nth-child(2) > a').click()

sleep(2)

datos_libros = {'Titulo' : [] , 
                'Autor(es)' : [] , 
                'Precio' : [] , 
                'Editorial' : [] , 
                'Idioma' : [] , 
                'Numero de paginas' : []}

for i in range (1 , 6):
    
    # Asigna a la variable "elemento" el libro que queremos encontrar
    elemento = driver.find_element('css selector' , f'#buscadorNF > div.col-md-9.col-12 > div > div.grid-view.mt-4 > div:nth-child({i}) > a > div > img')
    
    # Hace un scroll en la página hasta que se muestre el elemento
    driver.execute_script("arguments[0].scrollIntoView();", elemento)
    
    sleep(2)
    
    # Con el elemento a la vista, hace click para acceder a su página individual
    driver.find_element('css selector' , f'#buscadorNF > div.col-md-9.col-12 > div > div.grid-view.mt-4 > div:nth-child({i}) > a > div > img').click()

    sleep(2)
    
    # Agrega el título del libro al diccionario
    datos_libros['Titulo'].append(driver.find_element('css selector' , '#app > div.v-application--wrap > main > div > div > div > div:nth-child(3) > div > div.col-md-5.order-lg-2.col-12.order-1 > div > h1').text.lower().capitalize().strip())
    
    # Agrega los autores del libro al diccionario
    datos_libros['Autor(es)'].append(driver.find_element('css selector' , '#app > div.v-application--wrap > main > div > div > div > div:nth-child(3) > div > div.col-md-5.order-lg-2.col-12.order-1 > div > div.text-h5.d-flex.flex-wrap.author.mb-2.justify-center.justify-sm-start > div > a > span:nth-child(1)').text.lower().title().strip())
    
    # Agrega el precio del libro al diccionario (try-except por si el elemento está en otra posición)
    try:
        datos_libros['Precio'].append(float(driver.find_element('css selector' , '#app > div.v-application--wrap > main > div > div > div > div:nth-child(3) > div > div.border-left.col-md-4.col-12.order-3 > div > div:nth-child(3) > div.col.col-8 > strong').text.replace(" €" , "").replace("," , ".")))
    
    except:
        datos_libros['Precio'].append(float(driver.find_element('css selector' , '#app > div.v-application--wrap > main > div > div > div > div:nth-child(3) > div > div.border-left.col-md-4.col-12.order-3 > div > div:nth-child(4) > div > div.text-h4.font-weight-bold > span').text.replace(" €" , "").replace("," , ".")))
    
    # Asigna el elemento con la ficha técnica a una variable (try-except por si el elemento está en otra posición)
    try:
        ficha_tecnica = driver.find_element('css selector' , '#app > div.v-application--wrap > main > div > div > div > div:nth-child(6) > div > div.caracteristicas.mt-4.pt-4')
    except:
        ficha_tecnica = driver.find_element('css selector' , '#app > div.v-application--wrap > main > div > div > div > div:nth-child(7) > div > div.caracteristicas.mt-4.pt-4')
    
    # Hace scroll en la página hasta llegar a la ficha técnica
    driver.execute_script("arguments[0].scrollIntoView();", ficha_tecnica)
    
    sleep(2)
    
    # Captura los datos de la ficha técnica haciéndoles un split para convertirlos en lista
    try:
        datos_ficha = driver.find_element('css selector' , '#app > div.v-application--wrap > main > div > div > div > div:nth-child(6) > div > div.caracteristicas.mt-4.pt-4').text.split('\n')
    except:
        datos_ficha = driver.find_element('css selector' , '#app > div.v-application--wrap > main > div > div > div > div:nth-child(7) > div > div.caracteristicas.mt-4.pt-4').text.split('\n')

    
    # Agrega los elementos de la lista de datos_ficha según su índice:
    datos_libros['Editorial'].append(datos_ficha[3])
    datos_libros['Idioma'].append(datos_ficha[5].lower().capitalize())
    datos_libros['Numero de paginas'].append(int(datos_ficha[1]))
    
    # Vuelve a la página de resultados
    driver.back()
    
    sleep(2)
    
# Cierra el navegador    
driver.close()  

datos_libros

{'Titulo': ['El hijo olvidado',
  'Tres enigmas para la organización',
  'La paciente silenciosa',
  'Bajo tierra seca (premio nadal 2024)',
  'Las hijas de la criada (premio planeta 2023)'],
 'Autor(es)': ['Mikel Santiago',
  'Eduardo Mendoza',
  'Alex Michaelides',
  'Cesar Perez Gellida',
  'Sonsoles Onega'],
 'Precio': [21.75, 20.8, 10.4, 22.5, 21.75],
 'Editorial': ['B',
  'Seix Barral',
  'DEBOLSILLO',
  'Ediciones Destino',
  'Editorial Planeta'],
 'Idioma': ['Castellano',
  'Castellano',
  'Castellano',
  'Castellano',
  'Castellano'],
 'Numero de paginas': [576, 408, 400, 504, 480]}

In [4]:
pd.DataFrame(datos_libros)

,Titulo,Autor(es),Precio,Editorial,Idioma,Numero de paginas
0,El hijo olvidado,Mikel Santiago,21.75,B,Castellano,576
1,Tres enigmas para la organización,Eduardo Mendoza,20.80,Seix Barral,Castellano,408
2,La paciente silenciosa,Alex Michaelides,10.40,DEBOLSILLO,Castellano,400
3,Bajo tierra seca (premio nadal 2024),Cesar Perez Gellida,22.50,Ediciones Destino,Castellano,504
4,Las hijas de la criada (premio planeta 2023),Sonsoles Onega,21.75,Editorial Planeta,Castellano,480
